## Neo4j + Dependencies

In [ ]:
# Dependencies for later

!pip install neo4j
!pip install classy_classification
!pip install -U spacy
!python -m spacy download en_core_web_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 3.1 MB/s 
  Created wheel for neo4j: filename=neo4j-4.4.3-py3-none-any.whl size=116069 sha256=e231b7addc80c8de83eb2217cc69615698d25505ca79cc82410e3c17443b1b37
  Stored in directory: /root/.cache/pip/wheels/db/dd/76/acacd519878f133f2f869aec70db548d89e04013209c3c62bc
Successfully built neo4j
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 3.4 MB/s 
     |████████████████████████████████| 6.2 MB 13.7 MB/s 
     |████████████████████████████████| 4.2 MB 37.4 MB/s 
     |████████████████████████████████| 1.2 MB 65.6 MB/s 
     |████████████████████████████████| 86 kB 4.1 MB/s 
     |████████████████████████████████| 42 kB 693 kB/s 
     |████████████████████████████████| 660 kB 54.1 MB/s 
     |████████████████████████████████| 10.1 MB 16.2 MB/s 
     |

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 400.7 MB 5.2 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
class client_Node():

    def __init__(self, node_type, props):
        self.node_type = node_type
        self.props = props

    def getType(self):
        return self.node_type

    def getProps(self):
        return self.props

    def __str__(self):
        s = "Types: " + str(self.node_type) + "\n" + "Props: " + str(self.props)
        return s


In [ ]:
# Neo4J for knowledge graphs
#from client_Node import client_Node
class client_Edge():

    def __init__(self, n1, relType, n2):
        self.n1 = n1
        self.relType = relType
        self.n2 = n2

    def getNode1(self):
        return self.n1

    def getNode2(self):
        return self.n2

    def getRel(self):
        return self.relType

    def __str__(self):
        s = "Node 1\n" + str(self.n1) + "\n" + self.relType + "\n" + "Node 2\n" + str(self.n2)
        return s


In [ ]:
class DAOInterface:
    def __init__(self):
        pass

    def close(self):
        pass

    def query(self):
        pass

    def createNode(self):
        pass

    def createEdge(self):
        pass

    def getNode(self):
        pass

    def getEdge(self):
        pass

    def updateNode(self):
        pass

    def updateEdge(self):
        pass

    def deleteNode(self):
        pass

    def deleteEdge(self):
        pass


In [ ]:
from neo4j import GraphDatabase
import re

# For info on Cypher Query Language, see
# https://neo4j.com/developer/cypher/intro-cypher/

# Make sure to pip install neo4j


class Neo4jDAO(DAOInterface):
    def __init__(self, uri, user, pwd):
        super().__init__()
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)

    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

    def createNode(self, objType, dictArgs):
        # figure out a good way to replace the strings in the dictionary
        the_str = f"merge (x:{objType} {dictArgs})"
        the_str = re.sub("'(\w+)':", r"\1:", the_str)
        return self.query(the_str)

    def createEdge2(self, src_name, src_type, trg_name, trg_type, rel_type, two_way=False):
        the_str = f"MATCH (a:{src_type}), (b:{trg_type}) WHERE a.name = \'{src_name}\' AND b.name = \'{trg_name}\'" 
        the_str += f"CREATE (a)-[:{rel_type}]->(b)"
        if two_way:
            the_str += f"CREATE (b)-[:{rel_type}]->(a)"
        the_str += "RETURN *"

    def createEdge(self, objTypeN1, argsN1, objTypeN2, argsN2, relType):
        the_str = f"match (x:{objTypeN1} {argsN1}) match (y:{objTypeN2} {argsN2}) merge (x)-[:{relType}]->(y)"
        the_str = re.sub("'(\w+)':", r"\1:", the_str)
        return self.query(the_str)

    def getNode(self, objType, dictArgs):
        the_str = f"match (x:{objType} {dictArgs}) return x"
        the_str = re.sub("'(\w+)':", r"\1:", the_str)
        res = self.query(the_str)
        matches = []
        for x in res:
            n_type = [a for a in x[0].labels]
            n_props = [a for a in x[0].items()]
            node = client_Node(n_type, n_props)
            matches.append(node)
        return matches

    # Note from Chaz: I'm leaving this in, but it should be treated as deprecated
    def getConnections(self, objType, dictArgs):
        the_str = f"match (x:{objType} {dictArgs})--(y) return x, y"
        the_str = re.sub("'(\w+)':", r"\1:", the_str)
        return self.query(the_str)

    def getConnectionsWithRel(self, objType, dictArgs):
        the_str = f"match (x:{objType} {dictArgs})-[r]-(y) return x, r, y"
        the_str = re.sub("'(\w+)':", r"\1:", the_str)
        res = self.query(the_str)
        connections = []
        for x in res:
            n1_type = [a for a in x[0].labels]
            n1_props = [a for a in x[0].items()]
            n1 = client_Node(n1_type, n1_props)
            # print("Node ", n1.getType(), "\n", n1.getProps())
            rel_type = x[1].type
            n2_type = [a for a in x[2].labels]
            n2_props = [a for a in x[2].items()]
            n2 = client_Node(n2_type, n2_props)
            edge = client_Edge(n1, rel_type, n2)
            # print(edge)
            connections.append(edge)
        return connections

    def deleteNode(self, objType, dictArgs):
        the_str = f"match (x:{objType} {dictArgs}) detach delete x"
        the_str = re.sub("'(\w+)':", r"\1:", the_str)
        return self.query(the_str)

    def deleteEdge(self, objTypeN1, argsN1, objTypeN2, argsN2, relType):
        the_str = f"match (x:{objTypeN1} {argsN1})-[r:{relType}]-(y:{objTypeN2} {argsN2}) delete r"
        the_str = re.sub("'(\w+)':", r"\1:", the_str)
        return self.query(the_str)

    def updateNode(self, objType, dictArgs, update_name, update_data):
        the_str = f"match (x:{objType} {dictArgs}) set x.{update_name} = {update_data} return x"
        the_str = re.sub("'(\w+)':", r"\1:", the_str)
        return self.query(the_str)

    def updateEdge(self, objTypeN1, argsN1, objTypeN2, argsN2, relType, update_name, update_data):
        the_str = f"match (x:{objTypeN1} {argsN1})-[r:{relType}]-(y:{objTypeN2} {argsN2}) set r.{update_name} = {update_data} return r"
        the_str = re.sub("'(\w+)':", r"\1:", the_str)
        return self.query(the_str)


if __name__ == "__main__":
    pass


## Make Connection to Neo4J Database

In [ ]:
dao = Neo4jDAO(uri="neo4j+s://1ca9eb90.databases.neo4j.io", user="neo4j", pwd="gIWKGZDgAKfDs6nGBEpfsoJ67oEeRN9WRfBHi8jbxm4")

## Node creation etc.

In [ ]:
#dao.createNode("Object", {"name": "Water"})

dao.createEdge("Location", {"name": "Dark Woods"}, "Object", {"name": "Water"}, "contains")
dao.createEdge("Location", {"name": "Dark Woods"}, "Object", {"name": "Water"}, "contains")


[]

In [ ]:
dao.createEdge("Job", {"name": "Blacksmith"}, "Object", {"name": "Iron Sword"}, "can_make")
dao.createEdge("Job", {"name": "Blacksmith"}, "Object", {"name": "Gemstone Ring"}, "can_make")
dao.createEdge("Object", {"name": "Iron Sword"}, "Object", {"name": "Iron"}, "made_from")
dao.createEdge("Object", {"name": "Gemstone Ring"}, "Object", {"name": "Gemstone"}, "made_from")
dao.createEdge("Job", {"name": "Blacksmith"}, "Object", {"name": "Bronze Sword"}, "can_make")
dao.createEdge("Object", {"name": "Bronze Sword"}, "Object", {"name": "Bronze Ore"}, "made_from")

dao.createEdge("Job", {"name": "Carpenter"}, "Object", {"name": "Wood Bow"}, "can_make")
dao.createEdge("Object", {"name": "Wood Bow"}, "Object", {"name": "Yew Wood"}, "made_from")
dao.createEdge("Object", {"name": "Wood Bow"}, "Object", {"name": "Oak Wood"}, "made_from")
dao.createEdge("Object", {"name": "Wood Bow"}, "Object", {"name": "Pine Wood"}, "made_from")

dao.createEdge("Object", {"name": "Pine Wood"}, "Location", {"name": "Dark Woods"}, "found_in")
dao.createEdge("Object", {"name": "Oak Wood"}, "Location", {"name": "Dark Woods"}, "found_in")
dao.createEdge("Object", {"name": "Yew Wood"}, "Location", {"name": "Dark Woods"}, "found_in")

dao.createEdge("Object", {"name": "Gemstone"}, "Object", {"name": "Dragon"}, "is_guarded_by")
dao.createEdge("Object", {"name": "Iron"}, "Location", {"name": "The Mines"}, "found_in")
dao.createEdge("Object", {"name": "Iron"}, "Object", {"name": "Giant Spider"}, "is_guarded_by")
dao.createEdge("Object", {"name": "Bronze Ore"}, "Location", {"name": "The Mines"}, "found_in")
dao.createEdge("Object", {"name": "Bronze ore"}, "Object", {"name": "Giant Spider"}, "is_guarded_by")
dao.createEdge("Object", {"name": "Giant Spider"}, "Location", {"name": "The Mines"}, "lives_in")
dao.createEdge("Object", {"name": "Dragon"}, "Location", {"name": "The Cave of Yelat"}, "lives_in")

dao.createEdge("Object", {"name": "Steel"}, "Location", {"name": "The Mines"}, "found_in")

[]

In [ ]:
dao.createNode("Object", {"name": "Iron"})
dao.createNode("Object", {"name": "Steel"})
dao.createNode("Object", {"name": "Pine Wood"})
dao.createNode("Object", {"name": "Yew Wood"})
dao.createNode("Object", {"name": "Oak Wood"})
dao.createNode("Object", {"name": "Iron Sword"})
dao.createNode("Object", {"name": "Bronze Sword"})
dao.createNode("Object", {"name": "Wood Bow"})
dao.createNode("Object", {"name": "Gemstone Ring"})
dao.createNode("Object", {"name": "Bronze Ore"})
dao.createNode("Location", {"name": "The Mines"})
dao.createNode("Location", {"name": "The Cave of Yelat"})

[]

## WIP: Similarity matrix for cosine similiarity

In [ ]:
# Getting most similar matches through cosine similarity

from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Create the Document Term Matrix
count_vectorizer = CountVectorizer(stop_words='english')
count_vectorizer = CountVectorizer()
sparse_matrix = count_vectorizer.fit_transform(fact_str)

# OPTIONAL: Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(doc_term_matrix,
                  columns=count_vectorizer.get_feature_names())
df

NameError: ignored

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(df, df))


## Actual Pipeline

Step 1: Return all tuples from knowledge graph

(Definitely room to improve this, especially for when the graph gets big. Returns to the ever constant debate, should the knowledge graph have a world graph and character graph? It would be easier with both... ie. just query character graph when assigning a quest, to make sure it is consistant with what that character knows, but would complicate updating kg dynamically).

Step 2: Use a similarity metric to determine which tuples most closely match the prompt

(This area has the most potential for improvement - Do we get the number 1 similar ones then any connected nodes? I think that method would be pretty safe..)


Step 3: Take matches and verbalize into command.

  Step 3a. For each match (or just the one main match?) use classifier to determine if quest should be gathering, combat, or exploration.

  Step 3b. Get appropriate template for quest type.

  Step 3c. Fill in template according to values from templates.

In [ ]:
# Functions for step 1: extracting and processing information from kg

def verbalize_tuple(tuple_in):
  string = tuple_in[0].get_name() + " " + tuple_in[1] + " " + tuple_in[2].get_name()

  return string


def verbalize_tuple_2(tuple_in):
  string = tuple_in[0].get_properties()['name'] + " " + tuple_in[1] + " " + tuple_in[2].get_properties()['name']

  return string

def extract_tuples(record):
  tup_vals = []

  for i, val in enumerate(record):
    if i == 1:
      relationship_type = val.type
      relationship_type = relationship_type.replace('_', " ")

      tup_vals.append(relationship_type)
    else:
      node_name = None
      for name in val.labels:
        node_name = name

      node_properties = dict(val.items())
      node = Neo_Node(node_name, node_properties)

      tup_vals.append(node)

  return tuple(val for val in tup_vals)


def extract_facts(res):
  all_facts = []

  for record in res:
    new_tup = extract_tuples(record)
    all_facts.append(new_tup)

  return all_facts


def facts_to_strings(facts):
  fact_str = []

  for fact in facts:
    fact_str.append(verbalize_tuple_2(fact))

  return fact_str

# Class for storing nodes with properties and information with them.
class Neo_Node:
  def __init__(self, name, properties):
    self.name = name
    self.properties = properties

  def set_name(self, name):
    self.name = name

  def set_properties(self, properties):
    self.properties = properties

  def get_name(self):
    return self.name

  def get_properties(self):
    return self.properties

In [ ]:
# Functions for step 2: similarity metrics between input and kg
import difflib

def fuzzy_match(fact_strings, match_str):

    string = "Can you make me an iron sword please? What do you need from me?"

    # Get fuzzy matches (return best n=3)
    close_matches = difflib.get_close_matches(match_str, fact_strings, 1, 0.1)
    match_indices = []

    for match in close_matches:
      match_indices.append(fact_strings.index(match))

    print(match_indices)

    return close_matches[0], match_indices[0]

In [ ]:
# Functions/Classifier for determining quest type
import spacy
import classy_classification

data = {
    "exploration": ["Where should I go for an adventure?",
               "Any recommendations of places to visit?",
               "I am new here and looking for somewhere to stay the night.",
               "What are some famous locations in this area?",
               "I love traveling to foreign countries!",
               "I hope I can visit that city someday."],
    "combat": ["Do you need help cleaning things up around here?",
                "I'm just itching to fight some bad guys!",
                "Anybody causing you trouble recently?",
               "Looks like you have a pest problem on your hand.",
               "Anything that needs killing around here?",
               "It looks like the giant spiders are causing a problem in the town."],
    "gathering": ["Do you need me to get something?",
                  "Is there something I can get you?",
                  "Anything you are missing?",
                  "Looks like you need some supplies.",
                  "I should get you some iron to make a sword",
                  "What else do you need me to get?"]
}

nlp = spacy.load('en_core_web_lg')
nlp.add_pipe("text_categorizer",
    config={
        "data": data,
        "model": "spacy"
    }
)

#doc = nlp("Can you make me a sword please?")
#print(doc._.cats)

def get_quest_type(user_input):
  doc = nlp(user_input)

  result_dict = doc._.cats

  classified_quest_type = max(result_dict, key=result_dict.get)

  return classified_quest_type

In [ ]:
from neo4j.graph import Relationship
# Functions for step 3: turning selected tuples to command
import random

# Gathering template <start> <number> <item to find> <location>
# Exploration template <start> <location> <optional: purpose>
# Combat template <start> <number> <enemy> <optional: item to retrieve>
gathering_start_words = ['Find', 'Gather', 'Collect', 'Retrieve', 'Get me', 'Bring back', 'Obtain', 'Get']
exploration_start_words = ['Go to', 'Visit', 'Go see', 'Travel to', 'Journey to', ' Explore']
combat_start_words = ['Fight', 'Slay', 'Kill', 'Defeat', 'Vanquish', 'Eliminate']


relationship_mappings = {"made from": "to make"}


def create_command(best_tuple_string, classification, tuple_nodes):
  command_str = ""
  command_vals = []
  number_of_item = None

  # Get an appropriate command starting word
  if classification == 'gathering':
    start = random.choice(gathering_start_words)
    number_of_item = random.randint(1,9)
  elif classification == 'exploration':
    start = random.choice(exploration_start_words)
  else:
    start = random.choice(combat_start_words)
    number_of_item = random.randint(1,9)


  command_vals.append(start)

  if number_of_item is not None:
    command_vals.append(str(number_of_item))
    #command_str += str(number_of_item)

  print("Command string so far: ", command_vals)


  for tuple_val in reversed(tuple_nodes):
    node_val = ""

    if type(tuple_val) == Neo_Node:
      node_val = tuple_val.get_properties()['name']

      # TODO: get properties from node here

    else:
      node_val = tuple_val

      node_val = relationship_mappings[node_val]

    command_vals.append(node_val)


  print("Command string so far: ", command_vals)

  command_str = (' ').join(command_vals)

  return command_str

def create_simple_command(best_tuple_string, classification, tuple_nodes):
  command_str = ""
  command_vals = []
  number_of_item = None

  # Get an appropriate command starting word
  if classification == 'gathering':
    start = random.choice(gathering_start_words)
    number_of_item = random.randint(1,9)
  elif classification == 'exploration':
    start = random.choice(exploration_start_words)
  else:
    start = random.choice(combat_start_words)
    number_of_item = random.randint(1,9)

  command_vals.append(start)

  if number_of_item is not None:
    command_vals.append(str(number_of_item))
    #command_str += str(number_of_item)

  print("Command string so far: ", command_vals)


  for tuple_val in reversed(tuple_nodes):

    node_val = tuple_val.get_properties()['name']

    command_vals.append(node_val)

    break


  print("Command string so far: ", command_vals)


  return " ".join(command_vals)





In [ ]:
def find_connections(match_tuple, all_tuples):
  print("Length of tuples: ", len(all_tuples))

  # Save and return all relevant information in string
  all_relevant_information = []
  all_relevant_information.append(verbalize_tuple_2(match_tuple))

  possible_tuples = []


  for tup in all_tuples:

    for node in tup:

      if type(node) == str:
        continue

      #if match_tuple[0].get_properties()['name'] == node.get_properties()['name']:
        #print("In coming connection found")

        #possible_tuples.append(tup)
      if match_tuple[2].get_properties()['name'] == node.get_properties()['name']:
        print("Outgoing connection found")

        possible_tuples.append(tup)

  print("All possible matches")
  for tup in possible_tuples:
    print(verbalize_tuple_2(tup))


In [ ]:
def verbalize_command(user_input):
  print("User said: ", user_input)

  # Step 1: Query database for all information tuples
  query_str = "match (n)-[r]->(m) return n,r,m"
  res = dao.query(query_str)

  # Get turn neo4j result into fact tuples in a list
  facts = extract_facts(res)

  print("Facts found: ", facts)


  fact_strings = facts_to_strings(facts)

  print("Facts to strings: ", fact_strings)

  # Step 2: Get best match from similarity metric of best tuple to build command around
  # TODO: Maybe use multiple metrics to choose the best match?
  matches, indices = fuzzy_match(fact_strings, user_input)
  print("Best matching tuple: ", matches)
  print("Index in list: ", facts[indices])

  # Step 3: Turn tuples into command
  find_connections(facts[indices], facts)

  #Step 3a: Get quest classification
  quest_type = get_quest_type(user_input)
  print("Quest classified as: ", quest_type)

  # Get command string
  command = create_simple_command(matches, quest_type, facts[indices])

  print("Final command: ", command)


  return command

In [ ]:
# Choose user string -> The input from the user that was classified as looking for a quest
user_in = "I want to kill a dragon"
#user_in = "Can you make me an iron sword please?

command_out = verbalize_command(user_in)

User said:  I want to kill a dragon
Facts found:  [(<__main__.Neo_Node object at 0x7f8a8d193650>, 'member of', <__main__.Neo_Node object at 0x7f8a8d193310>), (<__main__.Neo_Node object at 0x7f8a8d193910>, 'makes', <__main__.Neo_Node object at 0x7f8a961c9a10>), (<__main__.Neo_Node object at 0x7f8a961c9990>, 'needs killed', <__main__.Neo_Node object at 0x7f8a961c9e10>), (<__main__.Neo_Node object at 0x7f8aa5cd55d0>, 'needs', <__main__.Neo_Node object at 0x7f8a863e20d0>), (<__main__.Neo_Node object at 0x7f8a863e2350>, 'wants', <__main__.Neo_Node object at 0x7f8a863e2310>), (<__main__.Neo_Node object at 0x7f8a863e28d0>, 'wants', <__main__.Neo_Node object at 0x7f8a863e2890>), (<__main__.Neo_Node object at 0x7f8a863e2910>, 'visits', <__main__.Neo_Node object at 0x7f8a863e2950>), (<__main__.Neo_Node object at 0x7f8a863e2b90>, 'visits', <__main__.Neo_Node object at 0x7f8a863e2b10>), (<__main__.Neo_Node object at 0x7f8a863e2ad0>, 'lives in', <__main__.Neo_Node object at 0x7f8a863e2a90>), (<__ma

## Test with model

In [ ]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 5.4 MB/s 
     |████████████████████████████████| 212 kB 53.2 MB/s 
     |████████████████████████████████| 1.1 MB 40.6 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
     |████████████████████████████████| 140 kB 52.6 MB/s 
     |████████████████████████████████| 127 kB 42.8 MB/s 
     |████████████████████████████████| 144 kB 42.7 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 271 kB 61.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.5.1
    Uninstalling dill-0.3.5.1:
      Successfully uninstalled dill-0.3.5.1
ERROR: pip's dependency resolver does not currently take into account all t

In [ ]:
from transformers import pipeline

# Visit https://huggingface.co/models?sort=downloads&search=Dizzykong
generator = pipeline('text-generation', model='Dizzykong/gpt2-medium-final', tokenizer='Dizzykong/gpt2-medium-final')

Downloading:   0%|          | 0.00/967 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
#prompt = command_out + "<div>"
prompt = "Collect 10 iron ore found in The Ore Fields required for an iron sword<div>"

sequences = generator(prompt, max_length=200, num_return_sequences=1)
for sequence in sequences:
  print(sequence['generated_text'].split("<eos>")[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Collect 10 iron ore found in The Ore Fields required for an iron sword<div>Craft 30 iron ore from the ore field nearby.<div>I've finally figured it out, friend. If you're looking for another piece of armor, then find you something else to craft it out of. The ore is plentiful, and if you seek to master it, then I'm sure you'll find someone who can help. Just head northwest up the road and take this iron ore around the mine until you find something suitable. It will be able to be found deep deep in the ore field before your final task.


In [ ]:
#prompt = command_out + "<div>"
prompt = "Find 15 gold nuggets<div>"

sequences = generator(prompt, max_length=200, num_return_sequences=1)
for sequence in sequences:
  print(sequence['generated_text'].split("<eos>")[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Find 15 gold nuggets<div>Deliver a gold nug to Pashra. You may use the gashra's shop to purchase the items you wish.<div>
